In [ ]:
pip install quickchart.io

In [82]:
import requests
import datetime, pytz
from quickchart import QuickChart
import csv



APIKEY = '5e36a901e55365a92e191b17845f0b2d'
city='Bangalore'


#get latest data by city name
def get_latest_weather_data_by_city():
  url = f'https://api.openweathermap.org/data/2.5/weather?q={city} &appid={APIKEY}'
	#print(f"Getting data via {url}")
  r=requests.get(url)
  if r.status_code == 200:
    latest_json_data=r.json()
    print("Temprature -> ", latest_json_data['main']['temp']-273.15)
    print("Humidity -> ", latest_json_data['main']['humidity'])

    time_sunrise=datetime.datetime.fromtimestamp(int(latest_json_data['sys']['sunrise'])).strftime('%Y-%m-%d %H:%M:%S')
    time_sunset=datetime.datetime.fromtimestamp(int(latest_json_data['sys']['sunset'])).strftime('%Y-%m-%d %H:%M:%S')
    print("Sunrise -> ", time_sunrise)
    print("Sunset -> ", time_sunset)
  else:
    print("data not found")
#End of get_latest_weather_data_by_city


#get last 5 days data by city name
def get_weather_data_by_city():
  url = f'https://api.openweathermap.org/data/2.5/forecast?q={city}%20&appid={APIKEY}'
	#print(f"Getting data via {url}")
  r=requests.get(url)
  #return r.json()
  if r.status_code == 200:
    return r.json()
  else:
    return None

#end of get_weather_data_by_city

#capture requred data and store in csv file
def get_reqired_data(json_data):
  list=[]
  for i in range(0,len(json_data['list'])):
    new_dict={}
    new_dict['DateTime']=json_data['list'][i]['dt_txt']
    new_dict['temp']=json_data['list'][i]['main']['temp']
    new_dict['pressure']=json_data['list'][i]['main']['pressure']
    new_dict['humidity']=json_data['list'][i]['main']['humidity']
    new_dict['speed']=json_data['list'][i]['wind']['speed']

    list.append(new_dict)
  
  keys = list[0].keys()
  with open('data.csv', 'w', newline='')  as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writeheader()
      dict_writer.writerows(list)

#end of get_reqired_data

#generate the chart
def get_quick_chart( data_list , output_file):
  qc = QuickChart()
  qc.width = 2000
  qc.height = 500
  labels = []	#Declare to hold the x-axis tick labels
  weather_temp = []  #get the data labels
  weather_pressure=[]
  weather_speed=[]

  for index in range(0,len(data_list)):
    local_time = data_list[index][0]
    labels.append(local_time )

    weather_temp.append( round(float(data_list[index][1])-273.15 ,1) )
    #weather_pressure.append( round( json_data['list'][index]['main']['pressure'] ,1) )
    weather_speed.append( round( float(data_list[index][4]) ,1) )

    qc.config = """{ 
            type: 'line', 
            data: { 
              labels: """ + str( labels ) + """,
          datasets: [
                { 
                  backgroundColor: 'rgb(255, 99, 132)', 
                  data: """ + str( weather_temp) + """,
                  lineTension: 0.4,
                  fill: false,
                  label: 'Temprature',
                },
                
                { 
                  backgroundColor: 'rgb(255, 99, 132)', 
                  data: """ + str( weather_speed) + """,
                  lineTension: 0.4,
                  fill: false,
                  label: 'Wind Speed',
                }
               
              ],
            },
            options: { 
                    title: { display: true,  text: '5-Days/3 hours Weather Forecast' }, 
                    legend: { display: true},


                    scales: { yAxes: [ { scaleLabel: 
                              { display: true, labelString: 'Temperature Degrees Celcius' } } ],
                               "xAxes": [{
                                   scaleLabel: 
                                            { display: true, labelString: 'Timestamp' },
                            
                                  
                                }]
                              },

                    plugins: {
                            datalabels: {
                              display: true,
                              align: 'bottom',
                              backgroundColor: '#ccc',
                              borderRadius: 5
                            },
                        }
                  },
          }""" 
  print(qc.get_short_url()) 	#Print out the chart URL
  qc.to_file(output_file)	#Save to a file

#End of Chart method


#read csv and return list
def get_CSV_Data():
  file = open("data.csv")
  csvreader = csv.reader(file)
  header = next(csvreader)
  rows = []
  for row in csvreader:
      rows.append(row)
  return rows
  file.close()

if __name__ == '__main__':
  print("Getting Weather Data")

  get_latest_weather_data_by_city()
  #get data from API using city name
  json_data =  get_weather_data_by_city()
  if json_data !=None:

    

    #get required data in csv format
    get_reqired_data(json_data)

    #read csv to populate in chart
    data_list=get_CSV_Data()
    
    #creating line chart 
    get_quick_chart( data_list , 'DataChart.jpg' )
    
  else:
    print("data not found")




Getting Weather Data
Temprature ->  28.370000000000005
Humidity ->  59
Sunrise ->  2021-09-25 00:38:40
Sunset ->  2021-09-25 12:43:56
https://quickchart.io/chart/render/zf-062a90f2-9cd0-4527-be58-a7465fdb3fa8
